# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("weather.csv")
weather_df

,0,temp,lat,hum,cloud,wind,lon
0,tahoua,25.60,14.89,12.0,44.0,4.12,5.27
1,ushuaia,13.00,-54.80,47.0,20.0,5.10,-68.30
2,iqaluit,-30.00,63.75,74.0,5.0,1.50,-68.51
3,sungairaya,24.33,0.70,93.0,71.0,1.21,108.90
4,mataura,17.78,-46.19,74.0,3.0,1.79,168.86
...,...,...,...,...,...,...,...
584,kartuzy,3.19,54.33,93.0,33.0,3.60,18.20
585,tokamachi,4.38,37.13,86.0,65.0,3.78,138.77
586,gunnedah,27.00,-30.98,61.0,4.0,1.50,150.25
587,praya,22.00,14.92,73.0,91.0,9.80,-23.51


In [3]:
weather_clean = weather_df.dropna()
weather_clean

,0,temp,lat,hum,cloud,wind,lon
0,tahoua,25.60,14.89,12.0,44.0,4.12,5.27
1,ushuaia,13.00,-54.80,47.0,20.0,5.10,-68.30
2,iqaluit,-30.00,63.75,74.0,5.0,1.50,-68.51
3,sungairaya,24.33,0.70,93.0,71.0,1.21,108.90
4,mataura,17.78,-46.19,74.0,3.0,1.79,168.86
...,...,...,...,...,...,...,...
584,kartuzy,3.19,54.33,93.0,33.0,3.60,18.20
585,tokamachi,4.38,37.13,86.0,65.0,3.78,138.77
586,gunnedah,27.00,-30.98,61.0,4.0,1.50,150.25
587,praya,22.00,14.92,73.0,91.0,9.80,-23.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude in locations
# Fill NaN values and convert to float
#df['myCol'] = df['myCol'].astype('Int64')

locations = weather_clean[["lat", "lon"]]
humidity = weather_clean['hum']

# Plot Heatmap
humidity_map = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
humidity_map.add_layer(heat_layer)
# Display figure
humidity_map


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [7]:
#filterinfDataframe = dfObj[(dfObj['Sale'] > 30) & (dfObj['Sale'] < 33) ]
ideal_weather = weather_clean[(weather_clean['temp'] < 27) & (weather_clean['temp'] > 21) & (weather_clean['hum'] < 10) & (weather_clean['cloud'] == 0)] 
ideal_weather

,0,temp,lat,hum,cloud,wind,lon
175,kidal,22.28,18.44,9.0,0.0,3.19,1.41
292,taoudenni,23.00,22.68,9.0,0.0,4.10,-3.98
361,tessalit,21.67,20.20,8.0,0.0,3.26,1.01
364,nioro,26.31,13.35,8.0,0.0,2.29,-15.75
501,saint-louis,25.70,16.33,8.0,0.0,5.20,-15.00
520,velingara,26.38,13.15,8.0,0.0,1.02,-14.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:


hotel_df = ideal_weather["Hotel Name"]

hotel_df


175    
292    
361    
364    
501    
520    
Name: Hotel Name, dtype: object

In [16]:
#params = {
#    "location": "39.952583,-75.16522",  # philadelphia coords,
#    "rankby": "distance",
#    "type": "restaurant",
#    "key": gkey,

# params dictionary to update each iteration
params = { "location": "",
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    Lat = row["lat"]
    Lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{Lat},{Lon}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

        # print the name_address url, avoid doing for public github repos in order to avoid exposing key
        # print(name_address.url)

        # convert to json
    #name_address = name_address.json()
    name_address
        # print(json.dumps(name_address, indent=4, sort_keys=True))
        # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    #try:
        #weather_df.loc[index, 'Hotel Name'] = name_address['main']['temp']
            #hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    
    #except (KeyError, IndexError):
        #print("Missing field/result... skipping.")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map